### Projeto Intermediário

Luís Filipe Loureiro

#### Pipeline:


- Mesclar três indicadores técnicos para uma decisão de direção
- Se desejar, dos indicadores dados em aula (MACD, RSI e Bollinger Bands), você pode escolher apenas 1 para usar
- Cada indicador contribui com um sinal, você deve montar uma forma de juntá-los, explicando o processo
- Você tem que modular para o tamanho da alocação para refletir a indicação do modelo. Por exemplo, se pelo os três indicadores indicam compra, deve-se comprar mais
- Escolher um período entre 5 e 10 anos com dados diários e simular com 3 Ativos diferentes


### Introdução

Esse estudo tem como objetivo estruturar uma estratégia de trading baseada em três indicadores da bilblioteca pandas-ta. Por meio do backtesting com dados de 2 anos, é testado a estratégia e avaliado sua perfomance.

A estratégia criada, chamada de Mix Strategies, é baseada nos indicadores RSI, ADX e Vortex. A partir de cada indicador, é gerado um sinal; 1 indicando compra e -1 indicando venda. A consolidação de cada indicação é feita por meio de uma soma. Caso seja positiva, foi realizado a compra do ativo, em caso negativo, sua venda.

### Metodologia


Para a Mix Strategies, foram usados três estratégias Relative Strengh Index (RSI), Average Directional Movement (ADX) e Vortex.

##### Sobre o Relative Strengh Index (RSI):

O RSI é um oscilador de momentum que indica se o mercado está overbought ou oversold. Para tanto, seu valor varia de 0 a 100. Como é padrão [1], foi adotado como banda superior (compra) valores a cima de 90 e como inferior (venda) abaixo de 10. O RSI é calculado a partir da razão entre a média dos retornos positivos pela média dos retornos negativos. A cada novo valor de price - close, é analisado se ele é superior a banda compra  ou inferior a banda de venda, alocando, assim, um sinal 1 ou -1, respectivamente 

$$ RSI = 100 - 100 ( 1 + up / down) $$

##### Sobre o Average Directional Movement (ADX):

O ADX é um indicador que mede a força de um movimento do instrumento. Para tanto, calcula-se uma média da expansão do valor do ativo. Seu valor é dado em um range de 0 a 100, e indica a força desse movimento, tanto para subida como descida. Assim, foi utilizado o valor padrão da literatura de 25 [2]. A cada novo preço, analisa-se se a força da trend (ADX) é superior a 25. Caso positivo, o movimento é significativo o suficiente para haver um posicionamento. É medido se essa trend é positiva, DMP ou negativa DMN. 

Caso o DMP seja superior ao DMN, o movimento indica subida, e é assinalado um sinal 1. Caso contrário, DMN superior a DMP, o sinal será de -1.


##### Sobre o Vortex:

O indicador Vortex calcula dois valores de oscilação; um que indica um movimento positivo "VTXP" e outro negativo "VTXM". Para cada novo close, compara-se tais valores. Caso o VTXP seja superior ao VTXM, o sinal será de compra, 1, caso seja menor, de venda, -1.

O Vortex é calculado a partir dos valores de alta ("high") e baixa ("low"). Quanto maior for a distância entre o valor atual de alta com relação ao passado de baixa maior será o VTXP. Já para o VTXM, ele é dado pela diferença entre o valor atual de low, pelo passado de high. Assim, o Vortex nunca é neutro, ele sempre indica um bias de compra ou de venda [3]. 

##### Fusão dos sinais e dimencionamento do size da operação

Para os três indicadores foi adotado como tempo padrão 14 dias. A fusão dos sinais ocorre por uma soma. Cada indicador tem um sinal próprio que indica compra (1), venda (-1) ou neutro (0). Caso a soma seja um final positivo, a Mix Strategies realizará uma compra. Caso contrário, ela entrará vendido.

O siza da operação é dado pelo valor da soma. Caso a soma resulte em 3, será comprado 3 ativos, caso resulte em -2, entrará vendido em 2x o valor do ativo. Importante notar que a cada novo posicionamento, é zerado a posição.

### Resultados

Para o backtesting, o horizonte de dados tem como data de início 06/03/2018 e fim em 06/03/2020. A estratégia foi analisada para três empresas; Apple ("AAPL"), Microsoft ("MSFT") e Google ("GOOG"). 

O resultado foi positivo para a Apple. O retorno sobre o capital anualizado foi de 19% e um lucro bruto de 122 dólares. Interessante notar os valores de média de ganho ("Avg win trade") e média de perda ("Avg loss trade"). A perda média foi de e dólares, já o ganho de 6 dólares, ou seja, uma razão de 3x. Isso indica que a estratégia, quando erra, erra pouco, e quando acerta, acerta com um retorno maior.


Para a Microsoft o resultado foi negativo. O retorno sobre o capital anualizado foi de -15%, com uma perda bruta de -99 dólares. Apesar da média de perda por trade ter sido 2.5 dólares e média de ganho de 5.7 dólares, ou seja, uma razão positiva, o Hitting Ratio foi baixo, 36.51%, indicando um alto índice de erros.

Por fim, com relação ao Google o resultado também foi negativo, com um retorno anualizado de -26%. No período foram realizados 71 trades, destes, 43 trades tiveram prejuízo, resultando em um Hitting Ratio de 39%. A razão entre a média trades entre os com ganho e com perda foi de 1.2, evidenciando que foram muito próximos. Assim, mesmo com uma média maior entre os trades acertados, dado que errou-se muito mais, o resultado final foi negativo.

### Conclusão

Assim, foi criado um script em Python utilizando três indicadores de momento por meio da biblioteca pandas-ta. A soma dos sinais dos indicadores determinou se o algoritmo compraria ou entraria vendido. A análise dos resultados evidenciou a imprevisibilidade do modelo, apesar dele ter performado para a Apple, os resultados foram negativos com relação à Microsoft e Google.

Buscando melhorar o modelo, uma possibilidade seria adicionar stop loss e stop gain. Eles aumentariam o sharp da estratégia e potencialmente reduziriam tal imprevisibilidade. Entretanto, eles onerariam a estratégia, reduzindo o número de trades. 

Uma segunda possibilidade seria alterar a fusão dos sinais. Pode-se alterar o peso que cada indicador tem na decisão final, aumentando a contribuição daquele com melhor perfomance.

In [21]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import datetime
import random

# Para fazer o backtesting
from backtesting_v2 import evaluateHist, evaluateIntr, Strategy, Order

import pandas_ta as ta
ticker = ['AAPL', "MSFT", "GOOG"]

In [22]:
import yahoofinancials as yf

start_date = '2018-03-06'
end_date =  '2020-03-06'

i = 0
for n in range(0, len(ticker)):
    data = yf.YahooFinancials(ticker[i]).get_historical_price_data(start_date, end_date, 'daily')

    # Ler os dados do JSON
    raw = pd.DataFrame(data[ticker[i]]['prices']).dropna()
    # Converter a data para o tipo correto datetime
    raw['formatted_date'] = pd.to_datetime(raw['formatted_date'])
    # Indica a data como o índice de cada linha
    raw = raw.set_index('formatted_date')
    # Removendo as colunas que não interessam
    df = raw.iloc[:,1:]

    # Acertando a ordem das colunas
    df = df.reindex(columns=['open', 'high', 'low', 'close', 'adjclose', 'volume'])
    # Salvando o CSV
    df.to_csv('{}.csv'.format(ticker[i]))
    i += 1

In [14]:
df = pd.read_csv('{}.csv'.format(ticker[0]))
df.head()

,formatted_date,open,high,low,close,adjclose,volume
0,2018-03-06,44.477501,44.562500,44.032501,44.167500,42.105804,95154000
1,2018-03-07,43.735001,43.962502,43.567501,43.757500,41.714943,126814000
2,2018-03-08,43.869999,44.279999,43.767502,44.235001,42.170158,95096400
3,2018-03-09,44.490002,45.000000,44.347500,44.994999,42.894676,128740800
4,2018-03-12,45.072498,45.597500,45.052502,45.430000,43.309383,128828400


In [15]:
class MixStrategies(Strategy):

    def __init__(self):
        
        self.period = 14
        self.lband = 10
        self.uband = 90 
        
        # lista para os dados
        self.prices = []
        self.high = []
        self.low = []
        
        self.rsi_indicator = []
        self.adx_indicator = []
        self.vortex_indicator = []
    
        self.side = 0
        
        self.init_capital = 100
        self.avail_capital = self.init_capital
        
        
        self.stop_gain = 0.1  # 10% Stop Gain
        self.stop_loss = -0.05  # 5% Stop Loss
        self.position_price = None # preço da posição, última compra/venda
        
        self.begin = 1 # Variavel de controle de inicio
       


    def receive(self, event):
        
        high = event.price[1]
        low = event.price[2]
        price = event.price[3]
        
        self.high.append(high)
        self.low.append(low)
        self.prices.append(price)
        
        if len(self.prices) >= self.period + 1:
            
            close = pd.DataFrame({'close': self.prices})               
            df = pd.DataFrame({'high': self.high, 'low': self.low, 'close': self.prices})      
            
            # RSI
            rsi = close.ta.rsi(length=self.period)
            self.rsi_indicator.append([self.uband, rsi.iloc[-1], self.lband])
            
            # ADX
            adx = df.ta.adx(length=self.period) 
            self.adx_indicator.append(adx.iloc[-1]['ADX_14'])
            
            # Vortex
            vortex = df.ta.vortex(length=self.period)
            self.vortex_indicator.append([vortex.iloc[-1]['VTXP_14'], vortex.iloc[-1]['VTXM_14']]) # o primeiro é indicador de subida, o segundo de descida
            
            
            # Indicadores
            
            # RSI
            signal_rsi = 0
            if rsi.iloc[-1] < self.lband:
                signal_rsi = 1 
            elif rsi.iloc[-1] > self.uband:
                signal_rsi = -1
                        
            # ADX                
            signal_adx = 0
            if adx.iloc[-1]['ADX_14'] > 25: # Average Directional Index é comparado para ver a sua força
                if adx.iloc[-1]['DMP_14'] > adx.iloc[-1]['DMN_14']:
                    signal_adx = 1
                else:
                    signal_adx = -1
            else:
                signal_adx = 0
                
            # Vortex    
            signal_vortex = 0
            if self.vortex_indicator[-1][0] > self.vortex_indicator[-1][1]: # se o indicador de subida for maior que o de descida, compra
                signal_vortex = 1
            elif self.vortex_indicator[-1][0] < self.vortex_indicator[-1][1]:
                signal_vortex = -1
            
           
                
            signal_sum = signal_rsi + signal_adx + signal_vortex
             
            # Caso seja o primeiro preço, o preço de compra é o preco atual
            if self.begin:
                self.position_price = price
                self.begin = 0
                
              
            # se a posição do self.side (qnt comprado ou qnt vendido) é diferente da indicação atual            
            if self.side != signal_sum:
        
                # Primeiro tem que zerar a alocação antes de alocar propriamente pela posição                
                if self.side > 0: # se está comprado, vende
                    self.submit(self.id, Order(event.instrument, Order.S, self.side, 0))
                    
                elif self.side < 0: # se está vendido, compra
                    self.submit(self.id, Order(event.instrument, Order.B, -self.side, 0))
                    
                    
                # Alocações pela estratégia:
                if signal_sum > 0: # se a indicacao é de compra, comprará mais dependendo da intensidade
                    self.submit(self.id, Order(event.instrument, Order.B, signal_sum, 0))
                   
                elif signal_sum < 0:
                    self.submit(self.id, Order(event.instrument, Order.SS, signal_sum, 0))
                    
                
                if signal_sum != 0: # se houve reposicionamento
                    self.position_price = price
                
                
                self.side = signal_sum
            



modelMS = MixStrategies()
print(evaluateHist(modelMS, {'AAPL': 'AAPL.csv'}))

Gross Profit: $112.77
Gross Loss: $-52.57
Gross Total: $60.20

Number of trades: 44
Hitting Ratio: 40.91%
Number of profit trades: 18
Number of loss trades: 26
Average number of events per trade: 10.55

Max win trade: $24.46
Avg win trade: $6.26
Max loss trade: $-9.51
Avg loss trade: $-2.02
Avg all trades: $1.37
Win/Loss ratio: 3.10

Max Profit: $30.62
Max Profit High/Low: $35.91
Max Drawdown: $-14.33
Max Drawdown High/Low: $-17.91

Max Allocation: $155.95
Avg Allocation: $73.62
Max Cash Required (margin): $155.95

Gross Total: $60.20
Total Fees: $8.80
Total Taxes: $20.09
Net Total: $31.30

Gross Return: 46.52%
Average Return: 1.06%
Net Return: 9.23%
Net Return Avg Alocation: 42.52%

Number of days: 504
Initial Capital: $100.00
Risk Free Rate: 13.75% yearly/0.0511% daily
Total Carry: $10.50
Net Total + Carry: $41.81
Net Return Capital: 41.81%
Net Return Capital Yearly: 19.08%




In [16]:
modelMS = MixStrategies()
print(evaluateHist(modelMS, {'MSFT': 'MSFT.csv'}))

Gross Profit: $131.96
Gross Loss: $-99.34
Gross Total: $32.61

Number of trades: 63
Hitting Ratio: 36.51%
Number of profit trades: 23
Number of loss trades: 40
Average number of events per trade: 7.51

Max win trade: $47.61
Avg win trade: $5.74
Max loss trade: $-16.21
Avg loss trade: $-2.48
Avg all trades: $0.52
Win/Loss ratio: 2.31

Max Profit: $81.25
Max Profit High/Low: $96.66
Max Drawdown: $-17.98
Max Drawdown High/Low: $-28.27

Max Allocation: $332.35
Avg Allocation: $135.70
Max Cash Required (margin): $332.35

Gross Total: $32.61
Total Fees: $12.60
Total Taxes: $28.28
Net Total: $-8.27

Gross Return: 11.03%
Average Return: 0.18%
Net Return: -17.65%
Net Return Avg Alocation: -6.09%

Number of days: 504
Initial Capital: $100.00
Risk Free Rate: 13.75% yearly/0.0511% daily
Total Carry: $-19.74
Net Total + Carry: $-28.01
Net Return Capital: -28.01%
Net Return Capital Yearly: -15.15%




In [23]:
modelMS = MixStrategies()
print(evaluateHist(modelMS, {'GOOG': 'GOOG.csv'}))

Gross Profit: $58.28
Gross Loss: $-74.41
Gross Total: $-16.13

Number of trades: 71
Hitting Ratio: 39.44%
Number of profit trades: 28
Number of loss trades: 43
Average number of events per trade: 6.59

Max win trade: $7.41
Avg win trade: $2.08
Max loss trade: $-6.74
Avg loss trade: $-1.73
Avg all trades: $-0.23
Win/Loss ratio: 1.20

Max Profit: $15.93
Max Profit High/Low: $16.47
Max Drawdown: $-6.74
Max Drawdown High/Low: $-8.02

Max Allocation: $142.16
Avg Allocation: $73.96
Max Cash Required (margin): $142.16

Gross Total: $-16.13
Total Fees: $14.20
Total Taxes: $13.93
Net Total: $-44.26

Gross Return: -15.65%
Average Return: -0.22%
Net Return: -56.30%
Net Return Avg Alocation: -59.84%

Number of days: 504
Initial Capital: $100.00
Risk Free Rate: 13.75% yearly/0.0511% daily
Total Carry: $-1.15
Net Total + Carry: $-45.41
Net Return Capital: -45.41%
Net Return Capital Yearly: -26.11%




Referências:


- [1] - https://www.fidelity.com/learning-center/trading-investing/technical-analysis/technical-indicator-guide/RSI#:~:text=Description,and%20oversold%20when%20below%2030.



- [2] - https://www.fidelity.com/learning-center/trading-investing/technical-analysis/technical-indicator-guide/adx#:~:text=Average%20Directional%20Movement%20Index%20(ADX)&text=ADX%20stands%20for%20Average%20Directional,System%20developed%20by%20Welles%20Wilder.




- [3] - https://www.investopedia.com/articles/active-trading/072115/understand-vortex-indicator-trading-strategies.asp

In [25]:
print("EOF")

EOF
